<a href="https://colab.research.google.com/github/Naveenkumar-The-magician/speech-emotion-recognition/blob/main/Speech_Emotion_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Task - 4 : speech emotion recognition**

In [3]:
import os
Root = "/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio"
os.chdir(Root)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
ls

modelForPrediction1.sav  speech-emotion-recognition-ravdess-data/


In [6]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [7]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result=np.hstack((result, mel))
    return result

In [8]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [9]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [10]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [11]:
x_train

array([[-6.36445862e+02,  4.03178978e+01, -2.45225735e+01, ...,
         9.02314969e-06,  4.44445777e-06,  7.63565095e-06],
       [-5.97994080e+02,  6.97111664e+01, -3.59924150e+00, ...,
         5.10924838e-05,  3.76197931e-05,  1.15771909e-05],
       [-7.63019897e+02,  2.89148197e+01,  3.83984303e+00, ...,
         4.03930653e-06,  3.34952415e-06,  1.57445925e-06],
       ...,
       [-5.25743286e+02,  4.09113770e+01, -2.43286400e+01, ...,
         4.92031359e-05,  2.16022563e-05,  1.82957665e-05],
       [-5.81369202e+02,  2.99467869e+01, -1.02583866e+01, ...,
         5.32001373e-04,  3.43305088e-04,  1.38068717e-04],
       [-6.73341125e+02,  5.37553329e+01,  4.17565680e+00, ...,
         8.08223922e-06,  3.30686930e-06,  2.55810733e-06]])

In [12]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(579, 194)


In [13]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [14]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [15]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [16]:
#Predict for the test set
y_pred=model.predict(x_test)

In [17]:
y_pred

array(['happy', 'calm', 'calm', 'happy', 'disgust', 'calm', 'calm',
       'disgust', 'calm', 'calm', 'disgust', 'happy', 'fearful', 'calm',
       'happy', 'calm', 'happy', 'calm', 'calm', 'fearful', 'disgust',
       'calm', 'fearful', 'disgust', 'happy', 'happy', 'fearful',
       'disgust', 'fearful', 'calm', 'happy', 'happy', 'happy', 'calm',
       'disgust', 'fearful', 'happy', 'happy', 'happy', 'happy', 'calm',
       'fearful', 'happy', 'happy', 'happy', 'calm', 'fearful', 'calm',
       'happy', 'happy', 'disgust', 'calm', 'calm', 'calm', 'calm',
       'disgust', 'calm', 'fearful', 'calm', 'calm', 'calm', 'happy',
       'happy', 'calm', 'calm', 'fearful', 'fearful', 'happy', 'calm',
       'happy', 'happy', 'fearful', 'fearful', 'happy', 'fearful',
       'happy', 'disgust', 'happy', 'calm', 'happy', 'disgust', 'happy',
       'calm', 'calm', 'calm', 'disgust', 'calm', 'calm', 'fearful',
       'fearful', 'fearful', 'happy', 'happy', 'calm', 'happy', 'fearful',
       'happ

In [18]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 68.04%


In [19]:
from sklearn.metrics import accuracy_score, f1_score

In [20]:
f1_score(y_test, y_pred,average=None)

array([0.75213675, 0.59459459, 0.69230769, 0.65546218])

In [21]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,calm,calm
2,calm,calm
3,disgust,happy
4,disgust,disgust
5,disgust,calm
6,disgust,calm
7,disgust,disgust
8,calm,calm
9,calm,calm


In [22]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [23]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['calm'], dtype='<U7')

In [24]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450531e-01,
         1.43299656e+01,  3.33637071e+00, -2.54071975e+00,
        -4.05790949e+00, -1.07119999e+01, -7.29413939e+00,
         1.74018872e+00, -4.19064236e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803197e+00, -5.68717539e-01,
        -6.47753334e+00, -1.24320555e+00, -2.80542517e+00,
        -5.43635798e+00, -4.46875393e-01, -3.63516593e+00,
        -2.98372602e+00, -5.63902557e-01, -1.65101993e+00,
        -5.55944741e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821292e+00,
        -5.54154456e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577074e+00, -1.62457860e+00, -2.03990698e+00,
        -3.62910867e+00,  6.33734703e-01,  6.48760676e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345614e-01,
         6.60463989e-01,  7.00949013e-01,  7.31556356e-01,
         7.46921539e-01,  7.23420441e-01,  7.16119647e-0

In [26]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab_Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_10/03-01-01-01-02-01-10.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction[0]

'happy'